**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.optimizers import *
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten, Dropout

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function ```act``` should return the next action to do. It takes two parameters :
- The state $a$ the agent
- If we want to train the agent or not

#### If the training is already done
If the agent has already learned the policy, we just want to exploit the policy. By setting, ```Train = False```, the agent selects the best action possible from the state $s$.

#### If we want to train the agent
In this scenario, the agent doesn't know the best policy. To find the best policy, it can not try every set of action because they are usualy to many possiblities.
The only thing it can do to explore the environnement is to choose actions randomly. But doing only random actions is not the best choice to explore efficiently.

A compromise is to choose "good" actions most of the time (exploit) and choose random action sometimes (explore). Doing this, we are shure that we explore a region that give big rewards (not just a random region).

#### Role of $\epsilon$
$\epsilon$ is the variable that encode how much the agent is exploring the environnement.
It can be a good stategie to decrease its value over time.

At the beggining, the agent axplores a lot, i.e. big $\epsilon$ (because it does not know the environnement at all). Then $\epsilon$ gradually decrease to promote more explotation.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

``` python
epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:                    # Up
            if self.x == self.grid_size-3: # Signifie que l'on est sur le bord de la grille, on ne peut pas aller plus haut
                self.x = self.x-1          # Donc on descent
            else:
                self.x = self.x + 1
        elif action == 1:                  # Down
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:                  # Left
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:                  # Right
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0 # S'il y a déjà un bonus sur la case, pas de malus (donc moins de malus que de bonus)

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0 # On enlève la récompense pour la position initiale
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train= 30 # set small when debugging (11 is the minimum for the script to work properly)
epochs_test= 11 # set small when debugging (> 11)

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

#### Array ```position```
This array is use to encode the position of the mouse. It's an array of shape : $N+4 \times N+4$, where $N$ is the size of the board. On the border (2 first and 2 last indexes) the array is filled with ```-1``` value. It idicates that the mouse can't go there. all the other slots are filled with  ```0``` value.

The array is part of the state. When rashaped to : ```[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]``` it gives to the agent the possible mooves whith a vision of 2 slots. If the slots has a ```0``` value, he can moove there, if it's a ```-1``` he can not (border of the map).

#### Array ```board```
Thus array contain the rewards (bonus and malus). The value ```self.board[i,j]``` indicates the reward obtained by the mouse if it goes to this position. When the reward is collected, it is set to ```0``` (there is nothing more to collect on a cell which as already been visited).

This array is the second part of the state. It is reshaped in the same purpose.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0,4)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        # A chaque début de partie, l'environnement est réinitialisé
        state = env.reset()
        game_over = False
        
        # win et loose réprésentent les gains (ou perte) de l'agent au cours de la partie
        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state,train=False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
            
            env.get_frame(e)
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win - lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 7.0/17.0. Average score (-10.0)
Win/lose count 6.5/13.0. Average score (-8.25)
Win/lose count 13.0/12.0. Average score (-5.166666666666667)
Win/lose count 11.0/15.0. Average score (-4.875)
Win/lose count 10.0/10.0. Average score (-3.9)
Win/lose count 10.5/20.0. Average score (-4.833333333333333)
Win/lose count 4.0/8.0. Average score (-4.714285714285714)
Win/lose count 7.0/7.0. Average score (-4.125)
Win/lose count 12.5/16.0. Average score (-4.055555555555555)
Win/lose count 13.5/11.0. Average score (-3.4)
Win/lose count 6.0/7.0. Average score (-3.1818181818181817)
Final score: -3.1818181818181817


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{s'}\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\label{eq:loss} \tag{1}
\end{equation*}




#### (a)
To proove the first expression, we need to use the Law of total expectation :

$$ Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] $$
$$
Q^\pi(s,a)= r(s,a) +  E_{p^{\pi}}[\sum_{t=1}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] $$
$$
Q^\pi(s,a)= r(s,a) +  \gamma \sum_{s'}\sum_{a'} p(s'|s,a') E_{p^{\pi}}[\sum_{t=1}^{\infty}\gamma^{t-1}r(s_{t},a_{t})|s_{1}=s',a_{1}=a'] $$
$$
Q^\pi(s,a)= r(s,a) +  \gamma \sum_{s'}\sum_{a'} p(s'|s,a') E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s',a_{0}=a'] $$
$$
Q^\pi(s,a)= r(s,a) +  \gamma \sum_{s'}\sum_{a'} p(s'|s,a') Q^\pi(s',a') $$
$$
Q^\pi(s,a)= r(s,a) +  \gamma E_{(s',a')\sim p(.|s,a)}[Q^\pi(s',a')] $$
$$
Q^\pi(s,a)= E_{(s',a')\sim p(.|s,a)}[r(s,a) +\gamma Q^\pi(s',a')] $$

#### (b)
The proof of the second expression is pretty simple. We just have to realize that the maxium over $\pi$ taken on $Q^\pi(s,a)$ implies to also take the maximum over $a'$ when $Q^\pi(s,a)$ is define by induction (because the best policy choose to take the best action).

For more clarity, I chose to express $ Q^*(s,a) $ as in $\eqref{eq:redef_Q}$. I changed $a_t$ to $\pi(s_{t})$. It make the dependence to the policy more obvious.

\begin{equation*}
Q^*(s,a)=\max_{\pi}E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},\pi(s_{t}))|s_{0}=s,a_{0}=a]
\label{eq:redef_Q} \tag{2}
\end{equation*}


Now the demontration :

$$ Q^*(s,a)=\max_{\pi}Q^\pi(s,a) $$
$$ Q^*(s,a)=\max_{\pi}E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_{t},\pi(s_{t}))|s_{0}=s,a_{0}=a]$$
$$ Q^*(s,a)=\max_{a',\pi}E_{(s',a')\sim p(.|s,a)}[r(s,a) +\gamma Q^\pi(s',a')] $$
$$ Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')] $$

#### (c)
The objective is to find the best policy. A good way to find it is to take at each state $s$ the action $a$ define as follow :
$$ a = \underset{a'}{\text{argmax}}Q^{*}(s,a)$$
In other word, if the agent know $Q^{*}(s,a)$ (or an approximation of it), it knows the best policy. The problem becomes : How to find $Q^{*}(s,a)$ ? In the next questions, we will try to approximate $Q^{*}(s,a)$ thanks to neural network. In that case, minimizing the loss $\mathcal{L}(\theta)$ make $Q(s,a,\theta)$ converge to $Q^{*}(s,a,\theta)$.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.position = 0
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) < self.max_memory :
            self.memory.append(None)
        self.memory[self.position] = m
        self.position = (self.position + 1) % self.max_memory

    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train = True)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        # Je pense qu'il y a un oubli, si l'agent ne retourne pas d'action en foncion du réseau, il ne va jamais apprendre
        # On retourne l'argument qui rend maximum Q*(s,a). Donc l'action qui mène à priori à la plus grnde reward.
        return np.argmax(self.model.predict(s.reshape((1,5,5,self.n_state))))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))    # 5*5 pour le nombre de case que peut voir la souris 
                                                                        # n_state états car c'est comme ça qu'est réprésenté l'état (board et position + cases déjà visitées)
        target_q = np.zeros((self.batch_size, 4))       # Q a 4 composantes qui représente les 4 actions que peut effectuer l'agent
        
        for i in range(self.batch_size):
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            input_states[i] = s_
            
            # On cherche à approximer Q*(s,a) grâce à la formule (1)
            # Pour ce faire, on a besoin d'information sur les états s', suivant l'action a
            
            # On initialise target_q avec la connaissance que l'on a déjà
            target_q[i] = self.model.predict(s_.reshape((1,5,5,self.n_state)))[0]
            
            if game_over_:
                # Si le jeu est terminé, la récompense reçu est la récompense finale
                target_q[i, a_] = r_
            else:
                # Sinon on calcul target_q normalement
                # On ajoute de la connaissance pour l'action qu'on a effectué et pour laquelle on a eu du feeedback
                target_q[i,a_] = r_ + self.discount*np.max(self.model.predict(n_s_.reshape((1,5,5,self.n_state))))

        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("adam", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        # L'entrée du réseau neuronnal est un vecteur de dimention 1
        # Il faut donc applatir la matrice d'entrée.
        # Sa dimention est : 5 * 5 * self.n_state
        
        # La dimention de sortie doit être : 4
        
        # Le nombre de couche n'est pas précisé, j'ai mis 2 couches fully-connected (input-16-output)
        model = Sequential()
        model.add(Reshape((5*5*self.n_state,), input_shape=(5,5,self.n_state)))
        model.add(Dense(16))
        model.add(Activation('relu'))
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=1e-3, epsilon = 0.1, memory_size=2000, batch_size = 128)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/011 | Loss 0.0206 | Win/lose count 2.5/6.0 (-3.5)
Epoch 001/011 | Loss 0.0318 | Win/lose count 2.0/2.0 (0.0)
Epoch 002/011 | Loss 0.0213 | Win/lose count 3.5/5.0 (-1.5)


KeyboardInterrupt: 

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

For the CNN, I tryed with only one convolutional layers of size (3,3). The output is size (3,3,n_state). I should give a good idea on what action to do. I added 1 fully connected layer.

In [13]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(5,5, self.n_state)))
        model.add(Conv2D(16, (3, 3), activation='relu'))
        '''
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        '''
        model.add(Flatten())
        model.add(Dense(4, activation = "linear"))
        
        model.compile(adam(lr=lr, decay=1e-4), "mse")
        self.model = model

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=1e-3, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

KeyboardInterrupt: 

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [15]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 1.0/2.0. Average score (-1.0)


KeyboardInterrupt: 

In [ ]:
HTML(display_videos('cnn_test10.mp4'))

In [ ]:
HTML(display_videos('fc_test10.mp4'))

#### Issue with these algo
It seems that the algorithms are not exploring at all the environnnement. A good stategy is to go back and forth. It prenvents the agent from collecting negative rewards.

#### Influence of the temperature

#### How to fix the problem ?
A solution is to have an $\epsilon$-greedy policy. At the begenning of the learning, the agent explores a lot (random mooves). It helps to have an accurate évaluation of $Q^*$ for a lot of different situations. Then, as $\epsilon$ decrease, it exploit more. The estimation of $Q*$ can get more precise where it matters.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [16]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    
    # On défini un facteur multiplicatif
    epsilon_decrease = (1-agent.epsilon)/(env.max_time*epoch)
    agent.set_epsilon(1)
    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # On défini le nouvel epsilon (à la fin tout les epoch, il vaudra la valeur entrée initialement)
            agent.set_epsilon(agent.epsilon - epsilon_decrease)
            
            # The agent performs an action
            action = agent.act(state, train=True)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')


class EnvironmentExploring(Environment):   
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        Environment.__init__(self, grid_size=grid_size, max_time=max_time, temperature=temperature)
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        self.malus_position[self.x, self.y] = 0.1

    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""
        state, reward, game_over = Environment.act(self, action)
        
        # Modification de la reward pour prendre en compte le malus de revisiter un emplacement
        if train:
            reward -= self.malus_position[self.x, self.y]
        
        if self.malus_position[self.x, self.y] < 0.9 :
            self.malus_position[self.x, self.y] += 0.1

        # 3 "features" states instead of 2
        malus_position =  self.malus_position[self.x-2:self.x+3, self.y-2:self.y+3, np.newaxis] # restricting to seen cells
        state = np.concatenate((malus_position, state), axis=2)

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""
        # reset environement
        state = Environment.reset(self)

        # reset malus
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        self.malus_position[self.x, self.y] = 0.1

        malus_pos =  self.malus_position[self.x-2:self.x+3, self.y-2:self.y+3, np.newaxis]
        state = np.concatenate((malus_pos, state), axis=2)
        
        return state

In [17]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=1e-3, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
epochs_train = 10
train_explore(agent, env,epochs_train , prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/010 | Loss 0.0190 | Win/lose count 10.0/49.00000000000005 (-39.00000000000005)
Epoch 001/010 | Loss 0.0303 | Win/lose count 5.0/65.30000000000001 (-60.30000000000001)
Epoch 002/010 | Loss 0.0088 | Win/lose count 12.0/40.600000000000044 (-28.600000000000044)
Epoch 003/010 | Loss 0.0105 | Win/lose count 12.5/35.70000000000003 (-23.20000000000003)
Epoch 004/010 | Loss 0.0067 | Win/lose count 23.5/20.499999999999993 (3.000000000000007)
Epoch 005/010 | Loss 0.0111 | Win/lose count 12.5/37.10000000000005 (-24.60000000000005)
Epoch 006/010 | Loss 0.0191 | Win/lose count 17.0/26.400000000000013 (-9.400000000000013)
Epoch 007/010 | Loss 0.0205 | Win/lose count 17.5/24.40000000000002 (-6.90000000000002)
Epoch 008/010 | Loss 0.0157 | Win/lose count 20.0/20.699999999999996 (-0.6999999999999957)
Epoch 009/010 | Loss 0.0193 | Win/lose count 23.5/21.399999999999988 (2.100000000000012)


In [18]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 22.0/3.0. Average score (19.0)
Win/lose count 12.5/4.0. Average score (13.75)
Win/lose count 17.0/4.0. Average score (13.5)
Win/lose count 17.0/3.0. Average score (13.625)
Win/lose count 19.0/0. Average score (14.7)
Win/lose count 22.0/2.0. Average score (15.583333333333334)
Win/lose count 20.5/1.0. Average score (16.142857142857142)
Win/lose count 15.0/1.0. Average score (15.875)
Win/lose count 20.5/1.0. Average score (16.27777777777778)
Win/lose count 18.0/1.0. Average score (16.35)
Win/lose count 16.5/0. Average score (16.363636363636363)
Final score: 16.363636363636363


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

### Question
- Quel est la dichotomie à faire sur ```Game Over``` ?

***